In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import os
import DM2

In [ ]:
def to_numpy(file_path, filename):
    with open(file_path + filename, 'r') as file:
        data = file.readlines()
    data = [line.replace('\n', '') for line in data]
    data = [line.split() for line in data]
    data = np.array([list(map(float, line)) for line in data])
    return data

In [ ]:
# import data
file_path = os.getcwd() + '/data/'
file_train = "EMGaussian.data"
file_test = "EMGaussian.test"
train = to_numpy(file_path, file_train)
test = to_numpy(file_path, file_test)
data_set = {'train': train,
            'test': test}

In [ ]:
def lnpuq(ut, qt, mu, cov):
    r=np.log(multivariate_normal(mu[qt], sigma_square[qt]).pdf(ut))
    print('r= ' , r)
    return r

In [ ]:
def forward_backward(data, pi, mu, sigma_square, logA):
    # forward recursion
    K=np.shape(logA)[0]
    N = data.shape[0]
    logalpha = np.zeros((N,K))
    logalpha[0,:] = [np.log(pi[k]) + np.log(multivariate_normal(mu[k], sigma_square[k]).pdf(data[0])) \
                     for k in range(K)]
    
    #We use the logarithm technique seen in course  
    for n in range(1,N):
        for k in range(K):
            listalpha = np.array([logalpha[n-1,i] + logA[i,k] for i in range(K)])
            alphamax = np.max(listalpha)                  
            logalpha[n,k] = np.log(multivariate_normal(mu[k], sigma_square[k]).pdf(data[n])) + \
                            alphamax + logsumexp(listalpha-alphamax)
                                   
    # backward recursion
    logbeta = np.zeros((N,K))
    #We use the logarithm technique seen in course  
    for n in range(N-2,-1,-1):
        for k in range(K):
            listbeta = np.array([logbeta[n+1,i] + logA[k,i] + \
                                np.log(multivariate_normal(mu[i], sigma_square[i]).pdf(data[n+1])) \
                               for i in range(K)])
            betamax = np.max(listbeta)
            logbeta[n,k] = betamax + logsumexp(listbeta-betamax)
    
    return (logalpha, logbeta)

def log_zeta(data, logalpha, logbeta, mu, sigma_square, logA):
    # un-normalized loggamma i.e log(p(q_t,u_0,...,u_T))
    #We start by calculating logp(u_0,...,u_T) and we use the same technique of log by writing p(U)=sum over q_0 p(q_0,U)
    listnorm= logalpha[-1,:]
    maxn=np.max(listnorm)
    norm=maxn+logsumexp(listnorm-maxn)
    
    #it's a three dimenstional matrix , at each time we have a KxK matrix 
    N=np.shape(data)[0]
    K=np.shape(logA)[0]
    logzeta=np.zeros((N-1,K,K))
    
    for n in range(1,N):
        for k in range(K):
            logzeta[n-1,:,k]= [np.log(multivariate_normal(mu[k], sigma_square[k]).pdf(data[n])) + \
                                logalpha[n-1,i] + logbeta[n,k] + logA[i,k] - norm for i in range(K)]
    return logzeta

def log_gamma(logalpha, logbeta):
    # un-normalized loggamma i.e log(p(q_t,u_0,...,u_T))
    loggamma = logalpha+logbeta
    #let's calculate normalized loggamma i.e log(p(q_t/u_0,......,u_T))
    #We start by calculating logp(u_0,...,u_T) and we use the same technique of log by writing p(U)=sum over q_0 p(q_0,U)
    listnorm = loggamma[0,:]
    maxn = np.max(listnorm)
    norm = maxn + logsumexp(listnorm-maxn)
    loggamma_norm = loggamma - norm
    return loggamma_norm

def update_parameters(data, logalpha, logbeta, pi, mu, sigma_square, logA):
    N = data.shape[0]
    K = pi.shape[0]
    
    loggamma = log_gamma(logalpha, logbeta)
    logzeta = log_zeta(data, logalpha, logbeta, mu, sigma_square, logA)
    
    # estimate log transition matrix A
    for i in range(K):
        norm = logsumexp(logzeta[:,i,:])
        logA[i,:] = [logsumexp(logzeta[:, i, k]) - norm for k in range(K)]
        
    # estimate initial distribution pi, and mu, sigma:
    pi = np.exp(loggamma[0,:])
    for k in range(K):
        weights = np.array([1 / np.sum(np.exp(loggamma[:,k] - loggamma[n,k])) for n in range(N)])
        mu[k] = np.average(data, axis=0, weights=weights)
        # or use: mu[k] = np.multiply(weights.reshape(-1,1), data).sum(axis = 0)
        sigma_square[k] = (data - mu[k]).T.dot(np.multiply(weights.reshape(-1,1), \
                          (data - mu[k])))

    # calculate loglikelihood of complete data.
    term_1 = np.exp(loggamma[0,:]).dot(loggamma[0,:]) 
    term_2 = (np.exp(logzeta).sum(axis = 0) * logA).sum()        
    term_3 = sum(np.exp(loggamma[:,k]).dot(np.log(multivariate_normal(mu[k], sigma_square[k]).pdf(data))) \
                 for k in range(K))
    loglikelihood = term_1 + term_2 + term_3

    return (pi, mu, sigma_square, logA, loglikelihood)

In [ ]:
def HMM(data, n_groups, epsilon, n_iter_max, init_param):
    
    K = n_groups
    N = data.shape[0]
    
    # init
    old_pi = init_param['pi']
    old_mu = init_param['mu']
    old_sigma_square = init_param['sigma_square']
    old_logA = np.log(np.ones((K,K))/K)
    
    logalpha, logbeta = forward_backward(data, old_pi, old_mu, old_sigma_square, old_logA)
    new_pi, new_mu, new_sigma_square, new_logA, new_loglikelihood = \
            update_parameters(data, logalpha, logbeta, old_pi, old_mu, old_sigma_square, old_logA)
    relative_error = epsilon
    count = 0
    
    while abs(relative_error) >= epsilon and count <= n_iter_max:
        old_pi, old_mu, old_sigma_square, old_logA, old_loglikelihood = \
                        new_pi, new_mu, new_sigma_square, new_logA, new_loglikelihood
        logalpha, logbeta = forward_backward(data, old_pi, old_mu, old_sigma_square, old_logA)
        new_pi, new_mu, new_sigma_square, new_logA, new_loglikelihood = \
                        update_parameters(data, logalpha, logbeta, old_pi, old_mu, old_sigma_square, old_logA)
        relative_error = new_loglikelihood/old_loglikelihood-1
        count += 1
    
    print('Converge after', count, 'iterations')
        
    return (new_pi, new_mu, new_sigma_square, new_logA, new_loglikelihood)

In [ ]:
def get_clusters(data, pi, mu, sigma_square, logA):
    # implementation of Viterbi algo
    
    K = logA.shape[0]
    N = data.shape[0]    
    logalpha = np.zeros((N,K))
    qmaxalpha = np.zeros((N, K), dtype=int)
    for k in range(K):
        logalpha[0,k]=np.log(pi[k])+lnpuq(data[0,:], k, mu, sigma_square)
    print(logalpha[0,:])
    for n in range(1, N):
        for k in range(K):
            proba=[logA[i,k] + np.log(multivariate_normal(mu[i], sigma_square[i]).pdf(data[n-1])) + \
                                logalpha[n, i] for i in range(K)]
            logalpha[n,k] = max(proba)
            qmaxalpha[n,k]=np.argmax(proba)       
    
    
    
    q_max = np.zeros(N, dtype=int)
        
    q_max[N-1] = np.argmax(logalpha[N-1, :])
    
    for n in range(1, N):
        n_next = N - n
        q_max[n_next - 1] = qmaxalpha[n_next, q_max[n_next]]
    
    return q_max
    

In [ ]:
epsilon = 1e-5
n_iter_max = 100
K = 4
mu_EM, sigma_square_EM, pi_EM, _ = DM2.EM(data_set['train'], K, epsilon, 100, 5, 100, False)

init_param = {'pi': pi_EM,
             'mu': mu_EM,
             'sigma_square': sigma_square_EM}

pi, mu, sigma_square, logA, loglikelihood = HMM(data_set['train'], K, epsilon, n_iter_max, init_param)

In [ ]:
colors = ["orange", "blue", "brown", "pink"]

fig = plt.figure(figsize=(30,12))
ax = fig.add_subplot(1,2, 1)

# plot EM HMM
clusters_HMM = get_clusters(data_set['train'], pi, mu, sigma_square, logA)
for k,color in enumerate(colors):
    # generate the ellipsoid corresponding to the parameters
    points = np.random.multivariate_normal(mean=mu[k], cov=sigma_square[k], size=100000)
    x, y = points.T
    ax.plot(x, y, ls = '', marker = 'o', ms = 0.000001, color = 'white')
    DM2.plot_point_cov(points, nstd=2, alpha=0.5, color='green')

    # plot data in each group
    group = train[clusters_HMM == k]
    ax.plot(group[:,0], group[:,1], ls = '', marker = 'o', \
                ms = 5, color = color, label = 'cluster ' + str(k+1))

ax.set_title('EM HMM', fontsize = 18)
ax.set_xlim((min(train[:,0]) - 2, max(train[:,0]) + 2))
ax.set_ylim((min(train[:,1]) - 2, max(train[:,1]) + 2))
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)        
plt.legend(fontsize = 14)
plt.grid()

# plot EM GMM

matrix_likelihood = DM2.mixture_density(pi_EM, mu_EM, sigma_square_EM, train, K)
clusters_GMM = np.argmax(matrix_likelihood, axis = 1)

ax = fig.add_subplot(1,2, 2)
for k,color in enumerate(colors):
    # generate the ellipsoid corresponding to the parameters
    points = np.random.multivariate_normal(mean=mu_EM[k], cov=sigma_square_EM[k], size=100000)
    x, y = points.T
    ax.plot(x, y, ls = '', marker = 'o', ms = 0.000001, color = 'white')
    DM2.plot_point_cov(points, nstd=2, alpha=0.5, color='green')

    # plot data in each group
    group = train[clusters_GMM == k]
    ax.plot(group[:,0], group[:,1], ls = '', marker = 'o', \
            ms = 5, color = color, label = 'cluster ' + str(k+1))

ax.set_title('EM GMM', fontsize = 18)
ax.set_xlim((min(train[:,0]) - 2, max(train[:,0]) + 2))
ax.set_ylim((min(train[:,1]) - 2, max(train[:,1]) + 2))
for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(14)
for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)        
plt.legend(fontsize = 14)
plt.grid()

fig.savefig(os.path.join(os.getcwd(), 'Plot2'))
plt.close(fig)

In [1]:
def calculate_loglikelihood(data, logalpha, logbeta, pi, mu, sigma_square, logA):
    N = data.shape[0]
    K = pi.shape[0]
    
    loggamma = log_gamma(logalpha, logbeta)
    logzeta = log_zeta(data, logalpha, logbeta, mu, sigma_square, logA)
    
    # estimate log transition matrix A
    for i in range(K):
        norm = logsumexp(logzeta[:,i,:])
        logA[i,:] = [logsumexp(logzeta[:, i, k]) - norm for k in range(K)]

    # calculate loglikelihood of complete data.
    term_1 = np.exp(loggamma[0,:]).dot(loggamma[0,:]) 
    term_2 = (np.exp(logzeta).sum(axis = 0) * logA).sum()        
    term_3 = sum(np.exp(loggamma[:,k]).dot(np.log(multivariate_normal(mu[k], sigma_square[k]).pdf(data))) \
                 for k in range(K))
    loglikelihood = term_1 + term_2 + term_3

    return loglikelihood

In [ ]:
logalpha, logbeta = forward_backward(data_set['test'], pi, mu, sigma_square, logA)
loglikelihood = calculate_loglikelihood(data_set['test'], logalpha, logbeta, pi, mu, sigma_square, logA)